# Imports packages





In [ ]:
%matplotlib inline

import os
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import anndata as ad
import scanpy as sc
import seaborn as sns
import squidpy as sq
from matplotlib_scalebar.scalebar import ScaleBar
from tqdm import tqdm
from skimage.measure import block_reduce
import scipy
import tifffile
import string

# Assign cell types

In [ ]:
#Load in the bead coordinate file
coords = pd.read_csv('BeadLocationsForR.csv', header=0)
print(coords.shape)
coords.head(3)

In [ ]:
#Load in the cell type weight matrix generated by running RCTD.R
weights = pd.read_csv('RCTD_Cell_Type_Weights.csv', header=0, index_col = 0)
print(weights.shape)
weights.head(3)

In [ ]:
#Find the cell type with the highest weight for each Slide-seq bead
weights['Max'] = weights.idxmax(axis=1)
weights.head(3)

In [ ]:
weights = weights.reset_index()
weights = weights.rename(columns={'index': 'barcode', 'Max':'CellType'})
weights_sel = weights[['barcode', 'CellType']]
print(weights_sel.shape)
weights_sel.head(3)

In [ ]:
weights_coord = pd.merge(coords, weights_sel, on= 'barcode', how='inner')
print(weights_coord.shape)
weights_coord.head(3)

In [ ]:
#Load in the bead class file generated by running RCTD.R
celltype = pd.read_csv('RCTD_Cell_Type.csv')
celltype = celltype.rename(columns={'Unnamed: 0': 'barcode'})
print(celltype.shape)
celltype.head(3)

In [ ]:
#Remove Slide-seq beads that belong to the class 'reject' 
celltype_reject = celltype.loc[(celltype['spot_class'].isin(['reject']))]
weights_coord_sel = weights_coord[~weights_coord.barcode.isin(celltype_reject['barcode'].values)]
print(weights_coord_sel.shape)
weights_coord_sel.head(3)

In [ ]:
celltype_assign = weights_coord_sel[['barcode','CellType']]
celltype_assign.to_csv('Puck_RCTD_Cell_Type_Weights.csv')
print(celltype_assign.shape)
celltype_assign.head(3)

In [ ]:
meta = weights_coord_sel.drop(columns = ['UMI','Pseudotime','Normalized Pseudotime'])
meta.to_csv('Puck_RCTD_Cell_Type_and_Coords_Weights.csv')
print(meta.shape)
meta.head(3)

# Neighborhood enrichment analysis

In [ ]:
counts = pd.read_csv('MappedDGEForR.csv', header=0, index_col = 0)
print(counts.shape)
counts.head(3)

In [ ]:
meta = pd.read_csv('Puck_RCTD_Cell_Type_and_Coords_Weights.csv',header=0, index_col = 0)
print(meta.shape)
meta.head(3)

In [ ]:
coords = meta.drop(columns=['CellType'])
print(coords.shape)
coords.head(3)

In [ ]:
counts_rctd = counts[counts['barcode'].isin(coords['barcode'].to_list())]
counts_rctd = counts_rctd.set_index('barcode')
counts_rctd.to_csv('MappedDGEForR_RCTD_Weights.csv') 
print(counts_rctd.shape)
counts_rctd.head(3)

In [ ]:
#Generate anndata
genes = counts_rctd.columns.values
counts_sparse = scipy.sparse.csr_matrix(counts_rctd.to_numpy())
counts_sparse.eliminate_zeros()
adata = ad.AnnData(counts_sparse, obs=coords)
adata.obs_names = coords.barcode.values
adata.var_names = genes
adata.obsm["spatial"] = coords[coords.barcode.isin(adata.obs_names.values)][["x", "y"]].to_numpy()
adata.obs["cluster"] = meta.CellType.values

In [ ]:
adata

In [ ]:
#Plot cell types
sc.set_figure_params(dpi = 300, format = 'eps',figsize=(8, 8))
fig, ax = plt.subplots(1, 1)
sc.pl.spatial(
    adata,
    color="cluster",
    spot_size=30,
    ax=ax, 
    save = 'Puck_RCTD_All_CellType'
)

In [ ]:
# Get just the SPG cells (SPG1-4 for the mouse, SPG1-5 for the human)
adata_subset = adata[adata.obs['cluster'].isin(['SPG1',
                                                'SPG2',
                                                'SPG3',
                                                'SPG4',
                                                'SPG5'])].copy()

# Calculate squidpy co-occurrence
sq.gr.co_occurrence(adata_subset,'cluster')

In [ ]:
sc.set_figure_params(dpi = 300, format = 'eps',figsize=(8, 8))
fig, ax = plt.subplots(1, 1)
sc.pl.spatial(
    adata,
    color="cluster",
    groups = ['SPG1','SPG2','SPG3','SPG4','SPG5'],
    spot_size=40,
    ax=ax,  
    save = 'Puck_RCTD_SPG_Distribution'
)

In [ ]:
# Calculate neighborhood enrichment score
sq.gr.spatial_neighbors(adata_subset)
sq.gr.nhood_enrichment(adata_subset, cluster_key="cluster")
sq.pl.nhood_enrichment(adata_subset, cluster_key="cluster",
                      figsize=(8, 8),
                      cmap='inferno',
                      vmin =-6,
                      vmax=6,
                      legend=True,
                      dpi=300,
                      save="Neighborhood_enrichment_matrix_SPG.eps")
plt.show()

In [ ]:
adata.write('Puck_RCTD_Anndata.h5ad')